In [ ]:
import numpy as np  
import pandas as pd  
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_excel('Online Retail.xlsx')  
df.head()  

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,573415,21166,COOK WITH WINE METAL SIGN,1,2011-10-30 16:17:00,2.08,13607.0,United Kingdom
1,553156,22516,CHILDS GARDEN RAKE BLUE,1,2011-05-13 14:58:00,2.10,17841.0,United Kingdom
2,578065,23169,CLASSIC GLASS COOKIE JAR,1,2011-11-22 15:41:00,8.29,NaN,United Kingdom
3,540798,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,6,2011-01-11 12:11:00,3.75,17365.0,United Kingdom
4,575177,23319,BOX OF 6 MINI 50'S CRACKERS,1,2011-11-08 18:41:00,4.96,NaN,United Kingdom


In [3]:
# Take a sample of 40,000 rows
df = df.sample(n=40000)

# Save the sample as a new Excel file
df.to_excel('Online Retail.xlsx', index=False)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 39344 to 16394
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    40000 non-null  object        
 1   StockCode    40000 non-null  object        
 2   Description  39895 non-null  object        
 3   Quantity     40000 non-null  int64         
 4   InvoiceDate  40000 non-null  datetime64[ns]
 5   UnitPrice    40000 non-null  float64       
 6   CustomerID   29943 non-null  float64       
 7   Country      40000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 2.7+ MB


In [5]:
# stripping extra spaces in the description  
df['Description'] = df['Description'].str.strip()  

In [6]:
# Transaction at sweden   
df[df['Country'] == "Sweden"].groupby(['InvoiceNo', 'Description'])['Quantity'].sum()

InvoiceNo  Description                        
539338     ROUND SNACK BOXES SET OF 4 FRUITS        6
           SET 3 RETROSPOT TEA,COFFEE,SUGAR        24
546530     MINI PAINT SET VINTAGE                 576
547645     PAPER BUNTING RETROSPOT                 12
550277     POSTAGE                                  3
           RED RETROSPOT TAPE                      12
551278     CHILDRENS CUTLERY DOLLY GIRL             8
552957     PACK OF 12 TRADITIONAL CRAYONS         432
           RED METAL BOX TOP SECRET                 4
557939     SET OF 60 PANTRY DESIGN CAKE CASES      24
559145     PLASTERS IN TIN WOODLAND ANIMALS        96
           SET 12 KIDS  WHITE CHALK STICKS        288
561890     SET OF 12 FAIRY CAKE BAKING CASES       24
562600     PAPER BUNTING RETROSPOT                 12
564539     CHARLOTTE BAG APPLES DESIGN             10
           JAM JAR WITH GREEN LID                  12
           PACK OF 12 WOODLAND TISSUES             24
565287     SPOTTY BUNTING          

In [7]:
Basket_Sweden = (df[df['Country'] == "Sweden"]  
        .groupby(['InvoiceNo', 'Description'])['Quantity']  
        .sum().unstack().reset_index().fillna(0)  
        .set_index('InvoiceNo'))

In [8]:
Basket_Sweden

Description,BOX OF 6 MINI VINTAGE CRACKERS,BUNDLE OF 3 RETRO NOTE BOOKS,CHARLOTTE BAG APPLES DESIGN,CHILDRENS CUTLERY DOLLY GIRL,CINNAMON SCENTED VOTIVE CANDLE,JAM JAR WITH GREEN LID,MAGIC DRAWING SLATE CIRCUS PARADE,MINI JIGSAW SPACEBOY,MINI LIGHTS WOODLAND MUSHROOMS,MINI PAINT SET VINTAGE,...,SET OF 20 VINTAGE CHRISTMAS NAPKINS,SET OF 3 REGENCY CAKE TINS,SET OF 60 PANTRY DESIGN CAKE CASES,SET OF 72 RETROSPOT PAPER DOILIES,SPOTTY BUNTING,STORAGE TIN VINTAGE LEAF,TREASURE TIN GYMKHANA DESIGN,VINTAGE CHRISTMAS BUNTING,WOODEN BOX OF DOMINOES,WRAP RED VINTAGE DOILY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
539338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,576.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
550277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
551278,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
557939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
559145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
561890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def hot_encoder(x):  
    if(x<= 0):  
        return 0  
    if(x>= 1):  
        return 1  

In [10]:
Basket_Encoded = Basket_Sweden.applymap(hot_encoder)  
Basket_Sweden = Basket_Encoded  

In [11]:
# applying apriori algorithm for Sweden
freq_items = apriori(Basket_Sweden, min_support = 0.05, use_colnames = True)
freq_items

C:\Anaconda\envs\ML\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.05,(BOX OF 6 MINI VINTAGE CRACKERS)
1,0.05,(BUNDLE OF 3 RETRO NOTE BOOKS)
2,0.05,(CHARLOTTE BAG APPLES DESIGN)
3,0.05,(CHILDRENS CUTLERY DOLLY GIRL)
4,0.05,(CINNAMON SCENTED VOTIVE CANDLE)
5,0.05,(JAM JAR WITH GREEN LID)
6,0.05,(MAGIC DRAWING SLATE CIRCUS PARADE)
7,0.05,(MINI JIGSAW SPACEBOY)
8,0.05,(MINI LIGHTS WOODLAND MUSHROOMS)
9,0.05,(MINI PAINT SET VINTAGE)


In [12]:
# applying association rule
rules = association_rules(freq_items, metric ="lift", min_threshold = 1)  
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])  
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(SET OF 3 REGENCY CAKE TINS),(BUNDLE OF 3 RETRO NOTE BOOKS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
1,(BUNDLE OF 3 RETRO NOTE BOOKS),(SET OF 3 REGENCY CAKE TINS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
2,(SET OF 72 RETROSPOT PAPER DOILIES),(BUNDLE OF 3 RETRO NOTE BOOKS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
3,(BUNDLE OF 3 RETRO NOTE BOOKS),(SET OF 72 RETROSPOT PAPER DOILIES),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
4,(WOODEN BOX OF DOMINOES),(BUNDLE OF 3 RETRO NOTE BOOKS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
79,"(SET OF 3 REGENCY CAKE TINS, BUNDLE OF 3 RETRO...","(SET OF 72 RETROSPOT PAPER DOILIES, WOODEN BO...",0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
80,(WOODEN BOX OF DOMINOES),"(SET OF 72 RETROSPOT PAPER DOILIES, SET OF 3 ...",0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
81,(SET OF 72 RETROSPOT PAPER DOILIES),"(BUNDLE OF 3 RETRO NOTE BOOKS, SET OF 3 REGENC...",0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
82,(SET OF 3 REGENCY CAKE TINS),"(SET OF 72 RETROSPOT PAPER DOILIES, BUNDLE OF...",0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0


In [13]:
# When you have items with a lift value higher than 1, it indicates a positive association between the antecedent and consequent. This means that the presence of the antecedent increases the likelihood of the consequent occurring. 
# To leverage this information, you can consider the following actions:
# 1. Promote Related Items: If you have a lift value higher than 1 for a specific item pair, you can promote the consequent item when the antecedent item is purchased. For example, if the antecedent is "bread" and the consequent is "butter" with a lift value higher than 1, you can place ... them together or offer a discount when both items are purchased.
# 2. Cross-Selling Opportunities: Items with a high lift value can indicate potential cross-selling opportunities. For example, if you have a lift value higher than 1 between "shampoo" and "conditioner," you can promote the purchase of conditioner to customers who buy shampoo.
# 3. Bundling or Package Deals: Items with a high lift value can be bundled together as a package deal. This can encourage customers to purchase both items together, leading to increased sales and customer satisfaction.
# 4. Store Layout and Placement: Items with a high lift value can be strategically placed near each other in the store to encourage customers to purchase them together. This can improve the shopping experience and increase sales.
# 5. Targeted Marketing Campaigns: Use the lift values to identify target customer segments that are more likely to purchase the consequent item when the antecedent item is present. This allows you to tailor marketing campaigns and promotions to those specific segments.
# By leveraging the lift values, you can make informed decisions on product placement, promotions, and marketing strategies to maximize sales and customer satisfaction.

In [14]:
rules[rules['antecedents'].apply(lambda x: len(x) == 1)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(SET OF 3 REGENCY CAKE TINS),(BUNDLE OF 3 RETRO NOTE BOOKS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
1,(BUNDLE OF 3 RETRO NOTE BOOKS),(SET OF 3 REGENCY CAKE TINS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
2,(SET OF 72 RETROSPOT PAPER DOILIES),(BUNDLE OF 3 RETRO NOTE BOOKS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
3,(BUNDLE OF 3 RETRO NOTE BOOKS),(SET OF 72 RETROSPOT PAPER DOILIES),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
4,(WOODEN BOX OF DOMINOES),(BUNDLE OF 3 RETRO NOTE BOOKS),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
5,(BUNDLE OF 3 RETRO NOTE BOOKS),(WOODEN BOX OF DOMINOES),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
6,(JAM JAR WITH GREEN LID),(CHARLOTTE BAG APPLES DESIGN),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
7,(CHARLOTTE BAG APPLES DESIGN),(JAM JAR WITH GREEN LID),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
8,(CHARLOTTE BAG APPLES DESIGN),(PACK OF 12 WOODLAND TISSUES),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0
9,(PACK OF 12 WOODLAND TISSUES),(CHARLOTTE BAG APPLES DESIGN),0.05,0.05,0.05,1.0,20.0,0.0475,inf,1.0


In [15]:
# promote_products = rules[rules['antecedents'].apply(lambda x: len(x) == 1)]
# if 'antecedents' in promote_products.columns:
#     st.dataframe(promote_products['antecedents'])
   

In [ ]:
%%writefile streamlit_apriori.py

import streamlit as st
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import openpyxl
# Load the association rules data
df = pd.read_excel('Online Retail.xlsx')
df['Description'] = df['Description'].str.strip()

country = st.selectbox("Select a country", df['Country'].unique())

# Transaction for the selected country
Basket_Country = (df[df['Country'] == country]
                  .groupby(['InvoiceNo', 'Description'])['Quantity']
                  .sum().unstack().reset_index().fillna(0)
                  .set_index('InvoiceNo'))

def hot_encoder(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

Basket_Encoded = Basket_Country.applymap(hot_encoder)
Basket_Country = Basket_Encoded

# Applying apriori algorithm for the selected country
freq_items = apriori(Basket_Country, min_support=0.01, use_colnames=True)

# Applying association rule
rules = association_rules(freq_items, metric="lift", min_threshold=1)
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

# Create a Streamlit app
def main():
    st.title("Association Rules Analysis")
    st.subheader("Top Association Rules for {}".format(country))
    
    # Display the products to promote
    st.subheader("Products to Promote")
    promote_products = rules[rules['antecedents'].apply(lambda x: len(x) == 1)]
    promote_products = promote_products.rename(columns={'antecedents': 'products'})
    if 'products' in promote_products.columns:
        st.dataframe(promote_products['products'])
    else:
        st.write("No 'products' column found in promote_products DataFrame.")

    # Display the cross-selling opportunities
    st.subheader("Cross-Selling Opportunities")
    cross_sell_products = rules[rules['antecedents'].apply(lambda x: len(x) > 1)]
    cross_sell_products = cross_sell_products.rename(columns={'antecedents': 'products'})
    if 'products' in cross_sell_products.columns:
        pd.set_option('display.max_colwidth', None)
        st.dataframe(cross_sell_products['products'])    
    else:
        st.write("No 'products' column found in cross_sell_products DataFrame.")

if __name__ == "__main__":
    main()


In [ ]:
!streamlit run streamlit_apriori.py